# PPMI preprocessing

This notebook can be used to convert all raw data into the desired format for baseline training and MAGNN implementation. 

This notebook contains the following steps:
- [1. Use dataloader class](#use_dataloader_class)
- [2. Convert 4 data components into MAGNN readable format](#convert_to_maggn_format)
- [3. Create train-test split](#create_train_test_split)
- [4. Define metapaths and obtain metapath instances](#define_metapaths)

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import scipy
import pickle

from sklearn.model_selection import train_test_split
from utils.data import load_PPMI_data
from collections import ChainMap

import multiprocessing
from functools import partial

import utils.preprocess

import data_utils as du
from PPMI_dataloader import *

data_dir = du.find_data_dir('app')
dataloader_file = du.get_file_path(data_dir, 'class based structure', 'dataloaders', 'dataloader.p')
metapaths_metabolites_file = du.get_file_path(data_dir, 'MAGNN', 'preprocessed', 'PPMI_neighbor_pairs.p')
val_train_test_idx_file = du.get_file_path(data_dir, 'MAGNN', 'preprocessed', 'val_train_test_idx.npz')
metabolite_matcher_file = du.get_file_path(data_dir, 'class based structure', 'metabolite matching', 'matcher.p')

#MAGNN input file names
adjM_file = du.get_file_path(data_dir, 'MAGNN', 'preprocessed', 'adjM.npz')
features_proteins_file = du.get_file_path(data_dir, 'MAGNN', 'preprocessed', 'features_proteins.npz')
features_metabolites_file = du.get_file_path(data_dir, 'MAGNN', 'preprocessed', 'features_metabolites.npz')
node_types_file = du.get_file_path(data_dir, 'MAGNN', 'preprocessed', 'node_types.npy')
labels_file = du.get_file_path(data_dir, 'MAGNN', 'preprocessed', 'labels.npy')

<a id='use_dataloader_class'></a>
## 1. Use DataLoader class

This notebook builds on the `DataLoader`, `ExerciseMetabolomicsDataLoader` and `MetaboliteNameMatcher`  classes in `PPMI_dataloader.py`

#### Naive CCS method
The default implementation uses the naive method to obtain concentration change sign (CCS). With $\Delta$ as mean log2fold change across subjects per metabolite

   $\text{CCS}(\Delta) =
    \left\{ \begin{matrix}
     1 & p < \Delta \\ 
     0 & -p \leq \Delta \leq p \\ 
     -1 & \Delta < -p 
    \end{matrix}
    \right. $

#### Network pruning
The network pruning is implemented as visualized in the figure.
<div> <img src="images/pruning_the_network.png" width="800"/> </div>

In [2]:
calc_CCS_settings = {'method': 'naive', 
                     'p_value': 0.1}

allowed_ccs_values = [-1, 0, 1]

include_feature_protein = {'molecular_function':       True,
                           'biological_process':       True,
                           'protein_class':            True}

include_feature_metabolite = {'molecular_weight':      False,
                              'state':                 True,
                              'kingdom':               True,
                              'super_class':           True,
                              'class':                 False,
                              'direct_parent':         False,
                              'molecular_framework':   True,
                              'alternative_parents':   True,
                              'substituents':          True,
                              'external_descriptors':  False,
                              'cellular_locations':    True,
                              'biospecimen_locations': True,
                              'tissue_locations':      True,
                              'pathways':              False}

include_feature_category = {'metabolite': include_feature_metabolite,
                            'protein': include_feature_protein}

publication_names = ['millan']

dataloader = DataLoader(publication_names, include_feature_category, metabolite_matcher_file=metabolite_matcher_file, calc_CCS_settings=calc_CCS_settings, allowed_ccs_values=allowed_ccs_values)
dataloader.save_state()
dataloader.print_data_counts()

Number of metabolites in publication Millan 2020: 368
Number of unnamed metabolites in publication Millan 2020: 14
Number of metabolites in publication Millan 2020 after removing unnamed: 354
Number of metabolites with KEGG id, without HMDB id: 29
Number of metabolites with HMDB id and data: 325
Number of unique duplicated HMDB ids: 9
Number of removed rows becuase target value was not allowed: 0
Total number of metabolites in full PPMI: 20759
Total number of metabolites with CCS and not in PPMI: 82
Number of metabolites with CCS data in full PPMI: 261

Number of valid metabolites with CCS data in full PPMI (y): 228
Number of metabolite feature columns (X): 253


### Save MetaboliteMatcher object

In [4]:
matcher = dataloader.metabolite_matcher
du.dump_in_pickle(metabolite_matcher_file, matcher)

### Load saved Dataloader object

In [2]:
dataloader = du.read_from_pickle(dataloader_file)

<a id='convert_to_maggn_format'></a>
## MAGNN preprocessing: convert data from dataloader into MAGNN readable format

In [3]:
def get_CCS_class(CCS, CCS_classes):
    return CCS_classes[CCS]

CCS_classes = {-1: 'down',
                0: 'unchanged',
                1: 'up'}

CCS_labels_binary = {'down': 0,
                     'up': 1}

CCS_labels_multi_class = {'down': 0,
                          'unchanged': 1,
                          'up': 2}

types = {'metabolite': 0,
         'protein': 1}

PPMI_adjM = nx.convert_matrix.to_numpy_array(dataloader.PPMI_pruned)
PPMI_type_mask = np.array([types['metabolite'] if 'HMDB' in node else types['protein'] for node in dataloader.PPMI_pruned.nodes()], dtype='int32')

#0 for down and 1 for up (binary case)
#0 for down and 1 for unchanged and 2 for up (multi class case)
classes = dataloader.y.apply(get_CCS_class, args=(CCS_classes,))
multi_class_labels = classes.apply(get_CCS_class, args=(CCS_labels_multi_class,))
PPMI_labels = multi_class_labels

scipy.sparse.save_npz(adjM_file, scipy.sparse.csr_matrix(PPMI_adjM))
scipy.sparse.save_npz(features_metabolites_file, scipy.sparse.csr_matrix(dataloader.X))
scipy.sparse.save_npz(features_proteins_file, scipy.sparse.csr_matrix(dataloader.protein_features))
np.save(labels_file, PPMI_labels)
np.save(node_types_file, PPMI_type_mask)

In [9]:
n_proteins = pd.Series(PPMI_type_mask).value_counts()[types['protein']]
n_metabolites = pd.Series(PPMI_type_mask).value_counts()[types['metabolite']]

<a id='create_train_test_split'></a>
## Creating train-val-test split

In [188]:
def train_val_test_stats(PPMI_labels, train_idx, val_idx, test_idx):
    n_samples = len(PPMI_labels)
    df = pd.DataFrame([['Train', len(train_idx), len(train_idx)/n_samples*100], 
    ['Val', len(val_idx), len(val_idx)/n_samples*100],
    ['Test', len(test_idx), len(test_idx)/n_samples*100]], columns = ['Group', 'Size', '%']).set_index('Group').style.format({'Size': "{:.0f}", "%": "{:.1f}%"})
    df.index.name = None
    return df  

rand_seed = 15669114
train_idx, test_idx = train_test_split(np.arange(len(PPMI_labels)), test_size=.4, random_state=rand_seed)
train_idx, val_idx = train_test_split(train_idx, test_size=.4, random_state=rand_seed)
train_idx.sort()
val_idx.sort()
test_idx.sort()

np.savez(val_train_test_idx_file,
         val_idx=val_idx,
         train_idx=train_idx,
         test_idx=test_idx)

train_val_test_stats(PPMI_labels, train_idx, val_idx, test_idx)

,Size,%
Train,81,35.5%
Val,55,24.1%
Test,92,40.4%


<a id='define_metapaths'></a>
## Defining metapaths and searching for metapath instances

Note here the difference from biochemical pathways: metapath ≠ pathway

The formal definition of a metapath $m$ for graph $G(V, E)$ with node types $V \to T$ and edge types $E \to R$ for relationship type is characterized as:
    
$$
\begin{align}
    m = T_{1} \stackrel{R_{1}}{\longrightarrow} T_{2} \stackrel{R_{2}}{\longrightarrow} \ldots \stackrel{R_{l}}{\longrightarrow} T_{l+1}
\end{align}
$$
    
With $T=T_{1} \circ T_{2} \circ \cdots \circ T_{l}$ and $R=R_{1} \circ R_{2} \circ \cdots \circ R_{l}$ and $l$ the length of the metapath. 

This work uses 2 metapaths:
- Metabolite – Protein – Metabolite (MPM)  
- Metabolite – Protein – Protein – Metabolite (MPPM) 

The following figure should help explain the concept of a metapath and metapath instances:

<div> <img src="images/metapath_instance_explainer.png" width="800"/> </div>

In [87]:
def get_metapath_neighbor_pairs_PPMI(M, type_mask, metapaths):
    outputs = []
    for metapath in metapaths:
        print('Getting neighbor pairs for ', metapath)
        # consider only the edges relevant to the expected metapath
        mask = np.zeros(M.shape, dtype=bool)
        for i in range(len(metapath)-1):
            temp = np.zeros(M.shape, dtype=bool)
            temp[np.ix_(type_mask == metapath[i], type_mask == metapath[i + 1])] = True
            temp[np.ix_(type_mask == metapath[i + 1], type_mask == metapath[i])] = True
            mask = np.logical_or(mask, temp)
        partial_g_nx = nx.from_numpy_matrix((M * mask).astype(int))
        
        metapath_neighbor_paris = {}
        for source in (type_mask == metapath[0]).nonzero()[0]:
            if source == 0 or source % 5 == 0:            
                print('  Node', source)
            for target in (type_mask == metapath[-1]).nonzero()[0]:
                has_path = False
                single_source_paths = nx.single_source_shortest_path(
                    partial_g_nx, source, cutoff=len(metapath))
                if target in single_source_paths:
                    has_path = True

                if has_path:
                    shortests = [p for p in nx.all_shortest_paths(partial_g_nx, source, target) if
                                 len(p) == len(metapath)]
                    if len(shortests) > 0:
                        metapath_neighbor_paris[(source, target)] = shortests       
        
        outputs.append(metapath_neighbor_paris)
    return outputs

def find_metapaths_from_source(metapath, type_mask, partial_g_nx, source):
    metapath_neighbor_pairs_source = {}
#     if source == 0 or source % 5 == 0:            
    print(f'  Node {source}', )
    for target in (type_mask == metapath[-1]).nonzero()[0]:
        has_path = False
        single_source_paths = nx.single_source_shortest_path(
            partial_g_nx, source, cutoff=len(metapath))
        if target in single_source_paths:
            has_path = True

        if has_path:
            shortests = [p for p in nx.all_shortest_paths(partial_g_nx, source, target) if
                         len(p) == len(metapath)]
            if len(shortests) > 0:
                metapath_neighbor_pairs_source[(source, target)] = shortests       
    return metapath_neighbor_pairs_source

#To speed up the process the _MP version of the function calculates metapath instances parrallized
def get_metapath_neighbor_pairs_PPMI_MP(M, type_mask, metapaths):
    outputs = []
    for metapath in metapaths:
        print('Getting neighbor pairs for ', metapath)
        # consider only the edges relevant to the expected metapath
        mask = np.zeros(M.shape, dtype=bool)
        for i in range(len(metapath)-1):
            temp = np.zeros(M.shape, dtype=bool)
            temp[np.ix_(type_mask == metapath[i], type_mask == metapath[i + 1])] = True
            temp[np.ix_(type_mask == metapath[i + 1], type_mask == metapath[i])] = True
            mask = np.logical_or(mask, temp)
        partial_g_nx = nx.from_numpy_matrix((M * mask).astype(int))
        
        metapath_neighbor_paris = {}
        a_pool = multiprocessing.Pool()
        func = partial(find_metapaths_from_source, metapath, type_mask, partial_g_nx)
        result = a_pool.map(func, (type_mask == metapath[0]).nonzero()[0])  
        a_pool.close()
        a_pool.join()
        metapath_neighbor_paris = dict(ChainMap(*result))
        
        outputs.append(metapath_neighbor_paris)
    return outputs

def get_node_names(i_nodes):
    node_names = list(PPMI_pruned.nodes)
    return [node_names[i_node] for i_node in i_nodes]

In [10]:
#Defining MPM and MPPM metapaths
MPM_metapath = (types['metabolite'], types['protein'], types['metabolite'])
MPPM_metapath = (types['metabolite'], types['protein'], types['protein'], types['metabolite'])

PPMI_expected_metapaths = [
    [MPM_metapath, MPPM_metapath], #metabolite metapaths
    [] #protein metapaths
]

In [88]:
#Obtaining metapaths can take quite a bit of time, order of tens of minutes to hours, depending on the hardware specs
#The original MAGNN metapath instance calculater did not allow for the MPPM metapath, so it was adjusted here
type_i = 'metabolite'
PPMI_neighbor_pairs = get_metapath_neighbor_pairs_PPMI_MP(PPMI_adjM, PPMI_type_mask, PPMI_expected_metapaths[types[type_i]])
du.dump_in_pickle(metapaths_metabolites_file, PPMI_neighbor_pairs)
G_list = utils.preprocess.get_networkx_graph(PPMI_neighbor_pairs, PPMI_type_mask, types[type_i])
for G, metapath in zip(G_list, PPMI_expected_metapaths[types[type_i]]):
    filename = 'metapath_graph_' + '-'.join(map(str, metapath)) + '.adjlist'
    metapaths_metabolites_file = du.get_file_path(data_dir, 'MAGNN', 'preprocessed', filename)
    nx.write_adjlist(G, metapaths_metabolites_file)
all_edge_metapath_idx_array = utils.preprocess.get_edge_metapath_idx_array(PPMI_neighbor_pairs)
for metapath, edge_metapath_idx_array in zip(PPMI_expected_metapaths[types[type_i]], all_edge_metapath_idx_array):
    filename = 'edges_' + '-'.join(map(str, metapath)) + '_idx.npy'
    edges_file = du.get_file_path(data_dir, 'MAGNN', 'preprocessed', filename)
    np.save(edges_file, edge_metapath_idx_array)

Getting neighbor pairs for  (0, 1, 0)
  Node 0
  Node 8
  Node 16
  Node 24
  Node 32
  Node 40
  Node 48  Node 56

  Node 49
  Node 25
  Node 17
  Node 1
  Node 33
  Node 34
  Node 35
  Node 9
  Node 50
  Node 18
  Node 2
  Node 41
  Node 19
  Node 57
  Node 26
  Node 3
  Node 10
  Node 42
  Node 51
  Node 36
  Node 43
  Node 11
  Node 27
  Node 52
  Node 4
  Node 44
  Node 53
  Node 58
  Node 37
  Node 28
  Node 45
  Node 29
  Node 5
  Node 20
  Node 12
  Node 30
  Node 38
  Node 59
  Node 6
  Node 46
  Node 31
  Node 47
  Node 54
  Node 13  Node 39

  Node 64
  Node 21
  Node 72
  Node 7
  Node 60
  Node 14
  Node 80
  Node 81
  Node 82
  Node 55
  Node 15
  Node 65
  Node 61
  Node 83
  Node 73
  Node 88
  Node 74
  Node 66
  Node 84
  Node 67  Node 62

  Node 68
  Node 89
  Node 90
  Node 69
  Node 96
  Node 70
  Node 91
  Node 92
  Node 63
  Node 85
  Node 93
  Node 22
  Node 86
  Node 71
  Node 104
  Node 97
  Node 105
  Node 94
  Node 23
  Node 87
  Node 112
  Node 98
  Node 75

In [12]:
#The code below should be run for storing metapath instance information in MAGNN readable format
#This allows for the batch training
type_i = 'metabolite'
target_idx_list = np.arange(len(PPMI_labels))
for metapath in PPMI_expected_metapaths[types[type_i]]:
    filename = 'edges_' + '-'.join(map(str, metapath)) + '_idx.npy'
    edges_file = du.get_file_path(data_dir, 'MAGNN', 'preprocessed', filename)
    edge_metapath_idx_array = np.load(edges_file)
    target_metapaths_mapping = {}
    for target_idx in target_idx_list:
        target_metapaths_mapping[target_idx] = edge_metapath_idx_array[edge_metapath_idx_array[:, 0] == target_idx][:, ::-1]
    filename_new = 'edges_' + '-'.join(map(str, metapath)) + '_idx.pickle'
    edges_file_new = du.get_file_path(data_dir, 'MAGNN', 'preprocessed', filename_new)
    du.dump_in_pickle(edges_file_new, target_metapaths_mapping)